In [2]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


import rpy2
import rpy2.robjects.numpy2ri
from stldecompose import decompose
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
stats = importr('stats')
stl=stats.stl
ts =stats.ts

Using TensorFlow backend.


In [3]:
data= pd.read_csv("NewData/NN5_interpolated.csv",header=None)
predictions = pd.read_csv("NewData/theta_25_horg.csv",index_col=0,skiprows = [1])
print(data.shape)
print(predictions.shape)

(111, 791)
(111, 594)


In [4]:
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
#     depth=data.shape[2]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size+1-horizon):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def make_k_input(data,window_size,horizon):
    length = data.shape[0]
    output= np.zeros([length-window_size+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
#     print(str(extra))
    data = np.append(data,np.zeros([horizon-extra]))
#     print(data)
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_k_input(data,window_size,horizon):
    length = data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    
#     data = np.append(data,np.zeros([horizon-extra]))
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)    

In [5]:
def smape(val,preds,horizon):
    temp = np.abs(val-preds)
    temp1=np.abs(val)+np.abs(preds)
    
    smape = 200/horizon*np.sum(temp/temp1)

    return smape




def smape_nzr(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200.0 * np.mean(diff)

# Auto-Encoder

### Horizon =56        (56--->32)

In [7]:
theta_length = predictions.shape[0]

horizon=56
window_size=25
theta_all=np.zeros(0)
for i in range(theta_length):
    current_pred= np.asarray(predictions.iloc[i].dropna().values,dtype=float)
    theta_all=np.concatenate((theta_all,current_pred),axis=0)
theta_input = make_k_input(theta_all,window_size,horizon)
normalized_theta_input=np.zeros(theta_input.shape)
for j in range(theta_input.shape[0]):
    scaler = MinMaxScaler()
    scaler.fit(theta_input[j])
    normalized_theta_input[j]= scaler.transform(theta_input[j])
#######################################################################################
norm_reshaped_input=normalized_theta_input.reshape(normalized_theta_input.shape[0],56)
ncol = normalized_theta_input.shape[1] #56

encoding_dim = 32
input_dim = Input(shape = (ncol, ))
# Encoder Layers
encoded1 = Dense(48, activation = 'relu')(input_dim)
encoded2 = Dense(40, activation = 'relu')(encoded1)
encoded3 = Dense(encoding_dim, activation = 'relu')(encoded2)
# Decoder Layers
decoded1 = Dense(40, activation = 'relu')(encoded3)
decoded2 = Dense(48, activation = 'sigmoid')(decoded1)
decoded3 = Dense(ncol, activation = 'sigmoid')(decoded2)
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded3)
autoencoder.compile(optimizer = 'adam', loss = 'mse')

autoencoder.fit(norm_reshaped_input, norm_reshaped_input, epochs = 500, batch_size = 32, shuffle = False)
encoder = Model(inputs = input_dim, outputs = encoded3)
encoded_input = Input(shape = (encoding_dim, ))
auto_out = np.array(encoder.predict(norm_reshaped_input))

new_theta_input=auto_out.reshape(auto_out.shape[0],auto_out.shape[1],1)
autoencoder.save('NewMyModelCheckpoint/autoencoder_model.h5')
encoder.save('NewMyModelCheckpoint/encoder_model.h5')
np.savetxt('NewMyModelCheckpoint/new_theta_input.csv',auto_out, fmt='%1.3f',delimiter=',')

    

# Case 1 : Only Data
#### Horizon=56
#### Window Size=25

In [7]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    Final_smape1=np.zeros(data_length)
    for y in range(data_length):
                
        num_test=56
        horizon=56
        window_size=25
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        train = series_data[:-n_val]
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        
        scaler = MinMaxScaler()
        scaler.fit(series_data.reshape(series_data.shape[0],1))
        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        
        tf.compat.v1.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')

        

        branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.0001))(branch_3)
        

        model=Model(inputs=[input_data],outputs=net)
        if y==110:
            model.summary()
        callback = ModelCheckpoint(filepath='NewMyModelCheckpoint/NN5_01.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))


        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=12,shuffle=True, epochs=200,verbose=0)

        model.load_weights('NewMyModelCheckpoint/NN5_01.h5')
        pred=model.predict({'input_data':test_input})
        pred=scaler.inverse_transform(pred)#######################################################

        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape1[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('NewMyModelCheckpoint/prediction01.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/111)
print("SMAPE2 (NZR approach)")
print(smape2/111)
print("MSE1:")
print(mse1/111)
print (sum(Final_smape1)/111)    
#print(smape1)
#print(smape2)

2019-09-30 19:39:56.909977


 99%|█████████████████████████████████████████████████████████████████████████████▎| 110/111 [1:36:11<00:52, 52.61s/it]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 25, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 25, 64)            256       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 25, 64)            12352     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 25, 64)            12352     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_final (Dense)          (None, 56)                89656     
Total params: 114,616
Trainable params: 114,616
Non-trainable params: 0
_________________________________________________________________


100%|██████████████████████████████████████████████████████████████████████████████| 111/111 [1:37:06<00:00, 53.40s/it]


2019-09-30 21:17:03.368115
-----------------
SMAPE1: (DeepEX approach)
42.93679942858188
SMAPE2 (NZR approach)
42.936799428581864
MSE1:
89.4748424246508
42.93679942858188


# Case 2
#### Data + Expert Knowledge (Size 32)
#### Adding Expert Knowledge to Layer 1
#### 5 Layers: 32 / 64 / 64 / 64 / 64
#### Horizon =56  window size=25

In [8]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
                
        num_test=56
        horizon=56
        window_size=25
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        train = series_data[:-n_val]
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        
        scaler = MinMaxScaler()
        scaler.fit(series_data.reshape(series_data.shape[0],1))
        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        train_p = series_pred[:-n_val]                                        
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
        
        
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(train_pred[j])
            train_pred_norm[j]= scaler4.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(val_pred[j])
            val_pred_norm[j]= scaler4.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(test_pred[j])
            test_pred_norm[j]= scaler4.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])

##########################################################################################################################
        ncol = 56
        encoding_dim = 32
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('NewMyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))
        
        
        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
        
        

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.compat.v1.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,encoding_dim),name='input_pred')
        
        #encoded12 = Dense(8, activation = 'relu')(input_pred)
        #encoded22 = Dense(16, activation = 'relu')(encoded12)
        #encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(input_pred)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_21 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_2)
        branch_22 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_21)
        branch_3=Flatten()(branch_22)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.0001))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==110:
            model.summary()
        callback = ModelCheckpoint(filepath='NewMyModelCheckpoint/NN5_02.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=12,shuffle=True, epochs=200,verbose=0)

        model.load_weights('NewMyModelCheckpoint/NN5_02.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)#######################################################

        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('NewMyModelCheckpoint/prediction02.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/111)
print("SMAPE2 (NZR approach)")
print(smape2/111)
print("MSE1:")
print(mse1/111)
print (sum(Final_smape2)/111)    


2019-09-30 21:17:03.439995


  0%|                                                                                          | 0/111 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|▋                                                                               | 1/111 [01:22<2:31:28, 82.63s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  2%|█▍                                                                              | 2/111 [02:48<2:31:45, 83.54s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 21%|████████████████▎                                                              | 23/111 [32:26<2:03:02, 83.90s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 22%|█████████████████                                                              | 24/111 [33:46<2:00:10, 82.88s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 23%|█████████████████▊                                                             | 25/111 [35:08<1:58:06, 82.40s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 41%|███████████████████████████████▉                                             | 46/111 [1:05:47<1:31:41, 84.64s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 42%|████████████████████████████████▌                                            | 47/111 [1:07:09<1:29:34, 83.97s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 43%|█████████████████████████████████▎                                           | 48/111 [1:08:32<1:27:41, 83.51s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 62%|█████████████████████████████████████████████████                              | 69/111 [1:38:14<58:59, 84.26s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 63%|█████████████████████████████████████████████████▊                             | 70/111 [1:39:38<57:35, 84.28s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 64%|██████████████████████████████████████████████████▌                            | 71/111 [1:41:03<56:18, 84.46s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 83%|█████████████████████████████████████████████████████████████████▍             | 92/111 [2:10:54<27:06, 85.61s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 84%|██████████████████████████████████████████████████████████████████▏            | 93/111 [2:12:18<25:34, 85.24s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 85%|██████████████████████████████████████████████████████████████████▉            | 94/111 [2:13:43<24:08, 85.20s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 25, 1)        0                                            
__________________________________________________________________________________________________
input_pred (InputLayer)         (None, 32)           0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 25, 32)       128         input_data[0][0]                 
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 32)        0           input_pred[0][0]                 
__________________________________________________________________________________________________
concatenat

100%|██████████████████████████████████████████████████████████████████████████████| 111/111 [2:38:08<00:00, 86.06s/it]


2019-09-30 23:55:12.085980
-----------------
SMAPE1: (DeepEX approach)
40.60149984865918
SMAPE2 (NZR approach)
40.60149984865918
MSE1:
84.9626759751263
40.60149984865918


# Case 3
#### Data + Expert Knowledge (Size 32)
#### Adding Expert Knowledge to Layer 1
#### 3 Layers: 32 / 64 / 64
#### Horizon =56  window size=25

In [16]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    Final_smape3=np.zeros(data_length)
    
    for y in range(data_length):
    
                
        num_test=56
        horizon=56
        window_size=25
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        train = series_data[:-n_val]
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        
        scaler = MinMaxScaler()
        scaler.fit(series_data.reshape(series_data.shape[0],1))
        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        train_p = series_pred[:-n_val]                                        
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
        
        
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(train_pred[j])
            train_pred_norm[j]= scaler4.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(val_pred[j])
            val_pred_norm[j]= scaler4.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(test_pred[j])
            test_pred_norm[j]= scaler4.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])

##########################################################################################################################
        ncol = 56
        encoding_dim = 32
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('NewMyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))
        
        
        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
        
        

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.compat.v1.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,encoding_dim),name='input_pred')
        
        #encoded12 = Dense(8, activation = 'relu')(input_pred)
        #encoded22 = Dense(16, activation = 'relu')(encoded12)
        #encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(input_pred)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.0001))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==110:
            model.summary()
        callback = ModelCheckpoint(filepath='NewMyModelCheckpoint/NN5_03.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=12,shuffle=True, epochs=200,verbose=0)

        model.load_weights('NewMyModelCheckpoint/NN5_03.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler4.inverse_transform(pred)#######################################################

        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape3[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('NewMyModelCheckpoint/prediction03.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/111)
print("SMAPE2 (NZR approach)")
print(smape2/111)
print("MSE1:")
print(mse1/111)
print (sum(Final_smape3)/111)    


2019-09-29 22:48:58.896180


  0%|                                                                                          | 0/111 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  1%|▋                                                                               | 1/111 [00:53<1:37:27, 53.16s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
  2%|█▍                                                                              | 2/111 [01:46<1:36:52, 53.33s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 21%|████████████████▎                                                              | 23/111 [22:29<1:21:52, 55.82s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 22%|█████████████████                                                              | 24/111 [23:23<1:20:07, 55.26s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 23%|█████████████████▊                                                             | 25/111 [24:15<1:18:06, 54.50s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 41%|█████████████████████████████████▌                                               | 46/111 [43:34<57:53, 53.44s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 42%|██████████████████████████████████▎                                              | 47/111 [44:27<56:50, 53.29s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 43%|███████████████████████████████████                                              | 48/111 [45:22<56:34, 53.88s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 62%|█████████████████████████████████████████████████                              | 69/111 [1:03:33<36:07, 51.61s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 63%|█████████████████████████████████████████████████▊                             | 70/111 [1:04:24<35:15, 51.60s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 64%|██████████████████████████████████████████████████▌                            | 71/111 [1:05:16<34:24, 51.61s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 83%|█████████████████████████████████████████████████████████████████▍             | 92/111 [1:23:27<16:24, 51.83s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 84%|██████████████████████████████████████████████████████████████████▏            | 93/111 [1:24:18<15:27, 51.52s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 85%|██████████████████████████████████████████████████████████████████▉            | 94/111 [1:25:10<14:40, 51.82s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 25, 1)        0                                            
__________________________________________________________________________________________________
input_pred (InputLayer)         (None, 32)           0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 25, 32)       128         input_data[0][0]                 
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 32)        0           input_pred[0][0]                 
__________________________________________________________________________________________________
concatenat

100%|██████████████████████████████████████████████████████████████████████████████| 111/111 [1:39:56<00:00, 52.29s/it]


2019-09-30 00:28:55.773293
-----------------
SMAPE1: (DeepEX approach)
25.12128524140436
SMAPE2 (NZR approach)
25.121285241404358
MSE1:
40.719032121336774
25.12128524140436


In [6]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,56])
    final_test = np.zeros([data_length,56])
    Final_smape3=np.zeros(data_length)
    
    for y in range(data_length):

        num_test=56
        horizon=56
        window_size=25
        current_row =np.asarray(data.loc[y].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-num_test]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
        
        train = series_data[:-n_val]
        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        
        scaler = MinMaxScaler()
        scaler.fit(series_data.reshape(series_data.shape[0],1))
        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=series_d[-num_test:]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]             
        
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        val_input = x_val
        test_input = x_test

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-num_test]
        
        train_p = series_pred[:-n_val]                                        
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
        
        
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(train_pred[j])
            train_pred_norm[j]= scaler4.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(val_pred[j])
            val_pred_norm[j]= scaler4.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler4 = MinMaxScaler()
            scaler4.fit(test_pred[j])
            test_pred_norm[j]= scaler4.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])

##########################################################################################################################
        ncol = 56
        encoding_dim = 32
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('NewMyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))
        
        
        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
        
        

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.compat.v1.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,encoding_dim),name='input_pred')
        
        #encoded12 = Dense(8, activation = 'relu')(input_pred)
        #encoded22 = Dense(16, activation = 'relu')(encoded12)
        #encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(input_pred)
        

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.0001))(branch_3)
        

        model=Model(inputs=[input_data,input_pred],outputs=net)
        if y==110:
            model.summary()
        callback = ModelCheckpoint(filepath='NewMyModelCheckpoint/NN5_0377.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))


        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=12,shuffle=True, epochs=200,verbose=0)

        model.load_weights('NewMyModelCheckpoint/NN5_0377.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler1.inverse_transform(pred)#######################################################

        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test,num_test)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(num_test),current_test)
        Final_smape3[y]=AAA
        mse1+=mse(pred.reshape(num_test),current_test)
        pbar.update(1)

np.savetxt('NewMyModelCheckpoint/prediction0377.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/111)
print("SMAPE2 (NZR approach)")
print(smape2/111)
print("MSE1:")
print(mse1/111)
print (sum(Final_smape3)/111)    


2019-09-30 18:03:45.251152


  0%|                                                                                          | 0/111 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0930 18:03:45.403905  6892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 18:03:45.423374  6892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 18:03:45.430861  6892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0930 18:03:45.503244  6892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_b

 17%|█████████████▌                                                                 | 19/111 [16:17<1:15:36, 49.31s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 18%|██████████████▏                                                                | 20/111 [17:05<1:14:14, 48.95s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 19%|██████████████▉                                                                | 21/111 [17:51<1:12:21, 48.24s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 38%|██████████████████████████████▋                                                  | 42/111 [34:29<55:21, 48.14s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 39%|███████████████████████████████▍                                                 | 43/111 [35:18<55:06, 48.63s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 40%|████████████████████████████████                                                 | 44/111 [36:09<54:51, 49.13s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 59%|███████████████████████████████████████████████▍                                 | 65/111 [53:42<41:57, 54.73s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 59%|████████████████████████████████████████████████▏                                | 66/111 [54:41<41:51, 55.82s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 60%|████████████████████████████████████████████████▉                                | 67/111 [55:41<41:54, 57.15s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

 79%|██████████████████████████████████████████████████████████████▋                | 88/111 [1:15:51<21:15, 55.45s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 80%|███████████████████████████████████████████████████████████████▎               | 89/111 [1:16:42<19:52, 54.21s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
 81%|████████████████████████████████████████████████████████████████               | 90/111 [1:17:32<18:32, 52.96s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* c

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 25, 1)        0                                            
__________________________________________________________________________________________________
input_pred (InputLayer)         (None, 32)           0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 25, 32)       128         input_data[0][0]                 
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 32)        0           input_pred[0][0]                 
__________________________________________________________________________________________________
concatenat

100%|██████████████████████████████████████████████████████████████████████████████| 111/111 [1:36:11<00:00, 51.14s/it]


2019-09-30 19:39:56.853575
-----------------
SMAPE1: (DeepEX approach)
29.048224638516796
SMAPE2 (NZR approach)
29.048224638516796
MSE1:
58.24574479915663
29.048224638516796
